# Joining / Clamping data

In [20]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import banana_lib

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
doe = pd.read_csv("../../data/doe.csv", index_col=None)
doe

,doe_id,Material_ID,Blechdicke,Niederhalterkraft,Ziehspalt,Stempel_ID,Einlegeposition,Ziehtiefe,Breite,UG,OG,E,Material_Name,Rp0,Rp0.2,Rp100,Rp25,Rp50
0,1,3,1.01,410,2.4,3,-5,30,70.2,1.71,2.00,191.37245,DC04_1.00mm,138.226960,147.601859,534.002871,377.443009,449.528189
1,2,5,1.48,400,2.4,3,-4,50,70.2,1.71,2.00,71.68198,AA5083_1.50mm,235.160326,237.951493,384.123147,369.820589,383.041834
2,3,1,1.00,210,2.4,3,-1,70,70.2,1.71,2.00,164.62254,DC01_1.00mm,133.182632,147.101263,517.275855,385.773439,447.384736
3,4,2,1.19,390,2.4,3,5,30,70.2,1.71,2.00,200.03813,DC01_1.20mm,160.714935,174.535075,564.455438,424.931018,490.442223
4,5,1,1.00,360,1.6,2,3,30,71.8,1.14,1.33,164.62254,DC01_1.00mm,133.182632,147.101263,517.275855,385.773439,447.384736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,3,1.01,90,1.6,2,0,70,71.8,1.14,1.33,191.37245,DC04_1.00mm,138.226960,147.601859,534.002871,377.443009,449.528189
996,997,4,0.99,480,2.4,3,5,30,70.2,1.71,2.00,71.94184,AA5083_1.00mm,248.796491,251.315902,385.373316,371.774337,384.299873
997,998,5,1.48,490,2.4,3,3,50,70.2,1.71,2.00,71.68198,AA5083_1.50mm,235.160326,237.951493,384.123147,369.820589,383.041834
998,999,5,1.48,120,2.4,3,-3,30,70.2,1.71,2.00,71.68198,AA5083_1.50mm,235.160326,237.951493,384.123147,369.820589,383.041834


In [37]:
doe_clamping = pd.read_csv("../../data/doe_clamping.csv", index_col=None)
doe_clamping

#                  98.00
Oberblech_ID      390.00
Unterblech_ID     196.00
Spanner_1          -4.00
Spanner_2           2.00
Spanner_3           4.00
Spanner_4           5.00
Oberblech_MID       6.00
Unterblech_MID      2.00
Success             1.00
Oberblech_zt       50.00
Unterblech_zt      30.00
Oberblech_t         1.01
Unterblech_t        1.19
Name: 97, dtype: float64

In [39]:
workpath = "/Users/vitay/.CMVolumes/Fraunhofer/ML@Karoprod/MachineLearning/joining"
workpath_single = "/Users/vitay/.CMVolumes/Fraunhofer/ML@Karoprod/MachineLearning/springback_uvmap"
output_file = '../../data/joining.csv'

variables = ['u', 'v', 'pos', 'x', 'y', 'z', 'thickness', 'epseqpl', 'thinning']

df = pd.DataFrame(columns = ['doe_id'] + variables)
df.to_csv(output_file, mode='w', index=False)

for f in sorted(os.listdir(workpath)):
    if not f.endswith(".h5"):
        continue

    # Get index
    idx_string = f[-7:-3]
    design = int(idx_string)
    print(design)

    if design in [3333, 5555, 7777]:
        continue

    # Open the file
    h5filepath = os.path.join(workpath, f)
    cs = banana_lib.Banana.from_h5(design, h5filepath)

    top_part = int(cs.parameter['Oberblech_ID'])
    bot_part = int(cs.parameter['Unterblech_ID'])

    try:
        top_path = os.path.join(workpath_single, f"Springback_Design{top_part:04}_uv.h5")
        m_top = banana_lib.Model(top_part, top_path)
        m_top.fit() # init interpolators
        uv_top = m_top.nodes[['nid0', 'u', 'v']]
        uv_top.rename(columns={'nid0': 'nid'}, inplace=True)
        top_df = pd.merge(uv_top, cs.part_top.nodes, on="nid")
        top_df.drop(columns=['nid'], inplace=True)
        top_df.insert(2, "pos", 1)
    except:
        print("Skipping", design, "because", top_part, "does not exist.")
        continue

    try:
        bot_path = os.path.join(workpath_single, f"Springback_Design{bot_part:04}_uv.h5")
        m_bot = banana_lib.Model(bot_part, bot_path)
        m_bot.fit() # init interpolators
        uv_bot = m_bot.nodes[['nid0', 'u', 'v']]
        uv_bot.rename(columns={'nid0': 'nid'}, inplace=True)
        bot_df = pd.merge(uv_bot, cs.part_bot.nodes, on="nid")
        bot_df.drop(columns=['nid'], inplace=True)
        bot_df.insert(2, "pos", 0)
    except:
        print("Skipping", design, "because", bot_part, "does not exist.")
        continue

    df = pd.concat([top_df, bot_df], ignore_index=True)

    df.insert(0, "doe_id", design)

    df.to_csv(output_file, mode='a', header=False, index=False)
    

1
2
3
5
7
8
9
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
27
28
31
32
34
35
36
37
38
39
41
42
43
44
45
46
47
48
50
51
52
53
54
55
57
58
59
60
61
62
64
65
66
67
68
69
70
71
72
74
75
76
78
79
80
81
82
83
84
87
88
89
90
91
92
93
94
95
96
97
98
Skipping 98 because 196 does not exist.
99
100
101
103
105
107
108
109
110
111
112
113
Skipping 113 because 450 does not exist.
114
115
116
117
118
119
126
130
131
133
134
135
137
138
139
140
142
144
145
146
147
148
149
150
151
153
154
155
156
157
158
159
161
162
163
Skipping 163 because 325 does not exist.
164
165
166
169
171
Skipping 171 because 342 does not exist.
174
175
176
177
178
181
182
Skipping 182 because 728 does not exist.
183
184
185
186
187
188
189
190
192
194
196
197
198
199
200
201
202
203
204
207
209
210
211
212
213
214
215
216
217
218
219
220
221
223
225
228
230
231
232
234
235
237
238
239
240
241
242
243
245
247
250
251
252
255
258
259
261
Skipping 261 because 44 does not exist.
262
263
264
265
268
269
276
281
284
285
286
287
289